In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random

2) Un importante servicio de monitoreo de aplicaciones cloud, sufrió un
incidente en las últimas semanas, tras el cual está conduciendo una
investigación del impacto que continua teniendo en sus integraciones a
nivel de conectividad. Como analista externos, tu misión es:

a) Calcular las métricas sumarizadas 'handshake_failed',
'ssl_failed', 'tls_failed' para cada uno de los integration_id a
partir del timestamp '1592187602', momento en el cual comenzó el
incidente. El formato esperado del df salida es:


b) Indicar aquellas 5 integraciones que tuvieron la mayor cantidad de
errores 'handshake_failed' a partir del inicio del incidente. El formato
esperado del df de salida es: (integration_name,
tot_handshake_failed)
Considerando los siguientes dataframes en CSV:
- metrics.csv describiendo información de métricas obtenidas. El mismo
tiene el formato ('client_id', 'integration_id' ,'metric',
'timestamp', 'value') donde la columna 'metric' indica el nombre
de una métrica que se registra en un cierto momento ('timestamp') cuyo
valor se guarda en la columna 'value'.
- integrations.csv describiendo las integraciones posibles que da a sus
clientes el servicio de monitoreo. ('integration_id',
'data_provider', 'integration_name')

In [2]:
original_metricas = pd.DataFrame(columns = ['client_id','integration_id','metric','timestamp','value'])
client_id =  [1,2,3,4,5,6,7,8]
integration_id = [2017, 2017, 2016, 1999, 1999, 2015, 2015]
metric = ['handshake_failed', 'ssl_failed', 'tls_failed', 'mateo_puto', 'gabi_forro', 'bauti_gil']
timestamp = [200, 2000, 2800, 100, 100, 2500, 25]
value = [6969, 420, 860, 1050]

integraciones =  pd.DataFrame(columns = ['data_provider','integration_id','integration_name'])
data_provider = ['tuvijea', 'tutia', 'tuprima', 'tuhermana']
integration_name = ['a','d','b','c','d']


for i in range(500):
    original_metricas = original_metricas.append({
        'client_id': random.choice(client_id),
        'integration_id': random.choice(integration_id),
        'metric': random.choice(metric),
        'timestamp': random.choice(timestamp),
        'value': random.choice(value),
    }, ignore_index = True)
    if i % 2 == 0:
        integraciones = integraciones.append({
            'integration_id': random.choice(integration_id),
            'data_provider': random.choice(data_provider),
            'integration_name': random.choice(integration_name)
        }, ignore_index = True)
        

In [3]:
metricas = original_metricas[original_metricas['timestamp'] >= 200]
metricas

,client_id,integration_id,metric,timestamp,value
0,6,2017,bauti_gil,200,1050
2,7,1999,bauti_gil,200,6969
5,6,2017,gabi_forro,2800,1050
6,4,1999,bauti_gil,2500,6969
7,3,2016,gabi_forro,2000,1050
...,...,...,...,...,...
492,5,2015,handshake_failed,2800,6969
493,2,2016,bauti_gil,2800,420
497,1,1999,bauti_gil,2800,860
498,2,2016,tls_failed,200,420


In [4]:
metricas = metricas[(metricas['metric'] == 'handshake_failed') | (metricas['metric'] == 'ssl_failed') | (metricas['metric'] == 'tls_failed')]
metricas.head()

,client_id,integration_id,metric,timestamp,value
9,2,1999,tls_failed,2500,6969
10,3,1999,tls_failed,2000,1050
13,3,2015,tls_failed,2500,6969
19,7,1999,ssl_failed,2800,1050
24,6,2017,tls_failed,2500,860


In [5]:
#metricas = metricas.groupby(metricas['integration_id']).agg({'':'sum'})
metricas = metricas.pivot_table(index='integration_id', columns='metric', values='value', aggfunc='sum')
metricas

metric,handshake_failed,ssl_failed,tls_failed
integration_id,,,
1999,25988,38916,45845
2015,30817,18978,26407
2016,4470,18158,12299
2017,29177,25377,31716


In [6]:
metricas = metricas.reset_index().rename_axis( None,axis=1)
metricas

,integration_id,handshake_failed,ssl_failed,tls_failed
0,1999,25988,38916,45845
1,2015,30817,18978,26407
2,2016,4470,18158,12299
3,2017,29177,25377,31716


In [7]:
metricas = metricas.set_index(metricas['integration_id'])
metricas

,integration_id,handshake_failed,ssl_failed,tls_failed
integration_id,,,,
1999,1999,25988,38916,45845
2015,2015,30817,18978,26407
2016,2016,4470,18158,12299
2017,2017,29177,25377,31716


In [8]:
metricas.drop(columns='integration_id')

,handshake_failed,ssl_failed,tls_failed
integration_id,,,
1999,25988,38916,45845
2015,30817,18978,26407
2016,4470,18158,12299
2017,29177,25377,31716


b) Indicar aquellas 5 integraciones que tuvieron la mayor cantidad de errores 'handshake_failed' a partir del inicio del incidente. El formato esperado del df de salida es: (integration_name, tot_handshake_failed) Considerando los siguientes dataframes en CSV:

In [9]:
metricas2 = original_metricas[original_metricas['timestamp'] >= 200]
metricas2 = metricas2[metricas2['metric'] == 'handshake_failed']
metricas2.head()

,client_id,integration_id,metric,timestamp,value
26,5,1999,handshake_failed,2000,6969
31,6,2017,handshake_failed,2500,1050
35,2,2015,handshake_failed,2500,1050
46,4,2016,handshake_failed,200,860
48,3,2016,handshake_failed,2000,420


In [10]:
metricas2 = metricas2.groupby('integration_id').agg({'metric':'count'})
metricas2

,metric
integration_id,
1999,16
2015,14
2016,6
2017,14


In [14]:
metricas2 = metricas2.reset_index()
final = metricas2.merge(integraciones, on='integration_id', how='inner')
#final = final[['integration_name','metric']]
final.nlargest(5, 'metric')

,level_0,index,integration_id,metric,data_provider,integration_name
0,0,0,1999,16,tuhermana,a
1,0,0,1999,16,tutia,b
2,0,0,1999,16,tuvijea,a
3,0,0,1999,16,tutia,b
4,0,0,1999,16,tuprima,b


In [12]:
final

,integration_name,metric
0,a,16
1,b,16
2,a,16
3,b,16
4,b,16
...,...,...
245,d,6
246,b,6
247,c,6
248,c,6
